# Imports

In [6]:
#Imports
%matplotlib widget
import pandas as pd
import numpy as np
import csv
import matplotlib
import matplotlib.pyplot as plt
import scipy
import datetime
import networkx as nx
import misc
import math
import sys
import random
import SmallBucket
import BigBucket
import RandomWarmUp1
import xyz_RandomWarmUp2 as RandomWarmUp2
import StaticDynamic
import DcOrient
import time
import xyz_DcOrientRandomSimple as DcOrientRandomSimple
import DcSimple
import StaticSimple
import xyz_BigBucketSimple as BigBucketSimple
import xyz_SmallBucketSimple as SmallBucketSimple
import xyz_StaticRandom as StaticRandom
import pickle

# Data Generation

In [ ]:
G = misc.createRandomGraph(size=200, density=0.5, variation=0.4, maxDegree=None)
misc.graphInfo(G)

avg = misc.getAverageDegree(G)
variance = 0
degreeTuples = list(G.degree(list(G.nodes())))

for tuple in degreeTuples:
    variance += abs(avg - tuple[1])

variance = float(variance) / G.number_of_nodes()

variance

In [ ]:
# Get increasing update sequence
updates = misc.extractUpdates(G, 'random')
Ge = nx.Graph()
Ge.add_nodes_from(list(G.nodes()))

In [ ]:
# Get update stream
updates = misc.extractStream(G, 10000, 0.5, 'random')
Ge = G.copy()

# Graph Saving and Loading

In [ ]:
# Always define a name
name = 'SizeTestSmall'

In [ ]:
# Create graph data
nodes = list(Ge.nodes())
edges = list(Ge.edges())
updateEdges = list(update[0] for update in updates)
updateTypes = list(update[1] for update in updates)

ge = Ge.copy()
oldColoring = nx.coloring.greedy_color(ge)
newColoring = dict({})
staticColors = []
staticRecolors = []
for update in updates:
    if update[1]:
        ge.add_edge(update[0][0], update[0][1])
    else:
        ge.remove_edge(update[0][0], update[0][1])
    newColoring = nx.coloring.greedy_color(ge)
    staticColors.append(misc.numberOfColors(newColoring))
    staticRecolors.append(misc.numberOfRecolors(oldColoring, newColoring))
    oldColoring = newColoring

gd = misc.GraphData(name, nodes, edges, updateEdges, updateTypes, staticColors, staticRecolors)
totStaticColors = gd.colorSum
totStaticRecolors = gd.recolorSum

In [ ]:
# Save graph data
misc.saveGraphData(gd)

In [ ]:
# Load graph data
gd = misc.loadGraphData(name)

Ge = gd.G.copy()

updates = gd.updates
staticColors = gd.staticColors
staticRecolors = gd.staticRecolors
totStaticColors = gd.colorSum
totStaticRecolors = gd.recolorSum

# Get final state of graph in local memory as well
G = gd.G.copy()
for update in updates:
    if update[1]:
        G.add_edge(update[0][0], update[0][1])
    else:
        G.remove_edge(update[0][0], update[0][1])

In [ ]:
print(str(gd.G.number_of_nodes())+' nodes, ' + str(gd.updateLength) + ' edges and estimated average C = ' + str(round(gd.colorSum/float(gd.updateLength))))
print('estimated average recolors = ' + str(round(gd.recolorSum/float(gd.updateLength), 2)))

In [ ]:
gd.staticColors[0]

# Algorithm Preparation

In [ ]:
# Only run once
experiments = []

In [ ]:
class Exp:
    def __init__(self, G, experimentList, name):
        self.G = G
        self.experimentList = experimentList
        self.name = name

In [ ]:
experimentList = []

experimentList.append([RandomWarmUp1.WarmUp1Algo(Ge, maxDegreeBound=None), 'Random Warm-Up'])

d = 5
experimentList.append([SmallBucket.SmallBucketAlgo(d, Ge), 'Small-Bucket algorithm (d = '+str(d)+')', 10])

d = 5
experimentList.append([BigBucket.BigBucketAlgo(d, Ge), 'Big-Bucket algorithm (d = '+str(d)+')', 10])

l = 10
experimentList.append([StaticDynamic.StaticDynamicAlgo(Ge, l, dynamicReset=False), 'Static-Dynamic algorithm (l = '+str(l)+')'])

l = 100
experimentList.append([StaticDynamic.StaticDynamicAlgo(Ge, l, dynamicReset=False), 'Static-Dynamic algorithm (l = '+str(l)+')'])

experimentList.append([DcOrient.DcOrientAlgo(Ge), 'DC-Orient'])

l = 10
experimentList.append([StaticSimple.StaticSimpleAlgo(Ge, l), 'Static-Simple algorithm (l = ' +str(l)+ ')'])

l = 100
experimentList.append([StaticSimple.StaticSimpleAlgo(Ge, l), 'Static-Simple algorithm (l = ' +str(l)+ ')'])

p = 0.8
experimentList.append([DcSimple.DcSimpleAlgo(Ge, p), 'DC-Simple (p = '+ str(p) + ')'])

p = 0.998
experimentList.append([DcSimple.DcSimpleAlgo(Ge, p), 'DC-Simple (p = '+ str(p) + ')'])

In [ ]:
experimentList = []

experimentList.append([RandomWarmUp1.WarmUp1Algo(Ge, maxDegreeBound=None), 'Random Warm-Up (local degrees)'])
experimentList.append([RandomWarmUp2.WarmUp2Algo(Ge, maxDegreeBound=None), 'Random Warm-Up 2 (local degrees)'])
experimentList.append([RandomWarmUp1.WarmUp1Algo(Ge, maxDegreeBound=209), 'Random Warm-Up (max degree)'])
experimentList.append([RandomWarmUp2.WarmUp2Algo(Ge, maxDegreeBound=209), 'Random Warm-Up 1 (max degree)'])



In [ ]:
experimentList = []
p = 0.0
experimentList.append([DcOrientRandomSimple.DcOrientRandomSimpleAlgo(Ge, p), 'DC-Orient without optimizations'])
experimentList.append([DcSimple.DcSimpleAlgo(Ge, p), 'DC-Orient with optimizations'])

In [ ]:
experiments.append(Exp(G, experimentList, name))

In [ ]:
for exp in experiments:
    print(exp.name)

# Experimenting

In [ ]:
def runExperiments(experiments, type='average'):

    validTypes = ['end', 'average', 'ratio']
    if type not in validTypes:
        print("Experiment type not valid, expected one of: " + str(validTypes))
        return


    for exp in experiments:
        
        G = exp.G
        experimentList = exp.experimentList
        name = exp.name

        # Load graph data
        gd = misc.loadGraphData(name)

        updates = gd.updates
        staticColors = gd.staticColors
        staticRecolors = gd.staticRecolors
        totStaticColors = gd.colorSum
        totStaticRecolors = gd.recolorSum

        # CSV Writer
        f = open('experiments/'+ str(name) + '.csv', 'w', newline='')
        writer = csv.writer(f)

        print('Graph used in the experiment:')
        misc.graphInfo(G)

        if (type == 'end'):
            C = staticColors[-1]
        elif (type == 'average'):
            C = round(float(totStaticColors)/len(updates), 2)
        elif (type == 'ratio'):
            C = 1
        row = [str(G.number_of_nodes()) + ' Nodes; ' + str(G.number_of_edges()) + ' Edges; C = ' + str(C),'Average nr. of Recolors', 'Average nr. of Colors', 'Time Taken (s)']
        writer.writerow(row)

        for experiment in experimentList:

            row = [str(experiment[1])]

            if len(experiment) == 3:
                random.seed(experiment[2])
            else:
                random.seed()

            print("")
            print("------------------------------------------------------------------")
            print(str(experiment[1]) + ":")
            iterator = misc.UpdateIterator(experiment[0], updates)

            recolors = 0
            colors = 0
            
            timer = 0
            verifytimer = 0
            colortimer = 0
            countertimer = 0
            
            while True:
                timestamp = time.perf_counter()
                # if not misc.verifyColoring(iterator.algo.G, iterator.algo.getColoring()):
                #     raise ValueError('The coloring produced is incorrect')
                verifytimer += time.perf_counter() - timestamp
                timestamp = time.perf_counter()
                c1 = iterator.algo.getColoring()
                if iterator.runUpdate(1):
                    timer += time.perf_counter() - timestamp
                    timestamp = time.perf_counter()
                    c2 = iterator.algo.getColoring()
                    colortimer += time.perf_counter() - timestamp
                    timestamp = time.perf_counter()
                    recolors += misc.numberOfRecolors(c1, c2)
                    colors += misc.numberOfColors(c2)
                    countertimer += time.perf_counter() - timestamp
                else:
                    #print("all updates finished")
                    break

            #timer = time.perf_counter() - timer

            print("")

            if (type == 'end'):
                # End values
                print('Results measured at the end')
                avgRecolors = float(recolors)/len(updates)
                print("Average number of recolors per update: " + str(avgRecolors))
                row.append(str(round(avgRecolors, 2)))
                finalColors = misc.numberOfColors(iterator.algo.getColoring())
                print("Final number of colors used: " + str(finalColors))
                row.append(str(finalColors))

            elif (type == 'average'):
                # Average values
                print("Resuts averaged over the entire update sequence")
                avgRecolors = float(recolors)/len(updates)
                print("Average number of recolors per update: " + str(avgRecolors))
                row.append(str(round(avgRecolors, 2)))
                avgColors = float(colors/len(updates))
                print("Average number of colors used: " + str(avgColors))
                row.append(str(round(avgColors, 2)))

            elif (type == 'ratio'):
                # Ratios
                print("Results given as a ratio")
                recolorRatio = float(recolors)/totStaticRecolors
                print("Ratio of recolors: " + str(recolorRatio))
                row.append(str(round(recolorRatio, 4)))
                colorRatio = float(colors)/totStaticColors
                print("Ratio of colors used: " + str(colorRatio))
                row.append(str(round(colorRatio, 4)))

            print("Time taken on algorithm : " + str(timer) + " seconds")
            row.append(str(round(timer, 3)))
            print("Time taken on verifying coloring: " + str(verifytimer) + " seconds")
            print("Time taken on counting colors and recolors: " + str(countertimer) + " seconds")
            print("Time taken on fetching the coloring: " + str(colortimer) + " seconds")
            # print("Average elementary operations : " + str(iterator.avgElemCounter))
            print("------------------------------------------------------------------")
            writer.writerow(row)

        print("End of experiment list reached")
        f.close()


In [ ]:
runExperiments(experiments, 'average')

# Graph Generation Experiments

In [ ]:
e = [['SmallBucket', 1, 30],
    ['BigBucket', 1, 30],
    ['StaticDynamic', 1, 200],
    ['StaticSimple', 1, 200],
    ['DcSimple', 0.4, 1],
    ['StaticRandom', 0.4, 1]]

iterations = 3

In [ ]:
e = [['BigBucket', 1, 30],
    ['BigBucketSimple', 0, 1, 2],
    ['BigBucketSimple', 0, 1, 5],
    ['BigBucketSimple', 0, 1, 10],
    ['BigBucketSimple', 0, 1, 20],
    ['BigBucketSimple', 0, 1, 30]]

iterations = 3

In [ ]:
coordinateLists = []
experimentListList = []

# Add a single iteration for the random warm ups
experimentList = []
tempList = []
for j in range(0, iterations):
    tempList.append([RandomWarmUp1.WarmUp1Algo(Ge, maxDegreeBound=None), 'Random Warm-Up'])
experimentList.append(tempList)
experimentListList.append(experimentList)

for exp in e:
    # Add different algorithms one by one by changing parameters
    experimentList = []
    algo = exp[0]
    min = exp[1]
    max = exp[2]
    r = 100            # Number of experiments per algorithm
    current = min
    lastCurrent = -100
    for i in range(0, r):
        if algo == 'SmallBucket':
            d = int(current)
            if d != int(lastCurrent):
                experimentList.append([[SmallBucket.SmallBucketAlgo(d, Ge), 'Small-Bucket algorithm (d = '+str(d)+')', 10]])
        elif algo == 'BigBucket':
            d = int(current)
            if d != int(lastCurrent):
                experimentList.append([[BigBucket.BigBucketAlgo(d, Ge), 'Big-Bucket algorithm (d = '+str(d)+')', 10]])
        elif algo == 'BigBucketSimple':
            d = exp[3] #temporary
            p = current
            experimentList.append([[BigBucketSimple.BigBucketSimpleAlgo(d, p, Ge), 'Big-Bucket Simple algorithm (p = '+str(p)+')']])
        elif algo == 'SmallBucketSimple':
            d = 10
            p = current
            experimentList.append([[SmallBucketSimple.SmallBucketSimpleAlgo(d, p, Ge), 'Small-Bucket Simple algorithm (p = '+str(p)+')']])
        elif algo == 'StaticDynamic':
            l = int(current)
            if l != int(lastCurrent):
                tempList = []
                for j in range(0, iterations):
                    tempList.append([StaticDynamic.StaticDynamicAlgo(Ge, l, dynamicReset=False), 'Static-Dynamic algorithm (l = '+str(l)+')'])
                experimentList.append(tempList)
        elif algo == 'StaticSimple':
            l = int(current)
            if l != int(lastCurrent):
                tempList = []
                for j in range(0, iterations):
                    tempList.append([StaticSimple.StaticSimpleAlgo(Ge, l=l), 'Static-Simple (l = '+str(l)+')'])
                experimentList.append(tempList)
        elif algo == 'DcSimple':
            p = current
            tempList = []
            for j in range(0, iterations):
                tempList.append([DcSimple.DcSimpleAlgo(Ge, p=p), 'DC-Simple (p = '+str(p)+')'])
            experimentList.append(tempList)
        elif algo == 'StaticRandom':
            p = current
            tempList = []
            for j in range(0, iterations):
                tempList.append([StaticRandom.StaticRandomAlgo(Ge, p=p), 'Static Random (p = '+str(p)+')'])
            experimentList.append(tempList)

        lastCurrent = current
        current = current + ((max-min)/100)

    experimentListList.append(experimentList)

# Append a single iteration of DC-Orient
experimentList = []
experimentList.append([[DcOrient.DcOrientAlgo(Ge), 'DC-Orient', 10]])
experimentListList.append(experimentList)

In [ ]:
# Print first experiment for each added algorithm
for experiments in experimentListList:
    print(experiments[0][0][1])

In [ ]:
def runGraphExperiments(experimentListList, name, type='average'):

    validTypes = ['end', 'average', 'ratio']
    if type not in validTypes:
        print("Experiment type not valid, expected one of: " + str(validTypes))
        return

    # Iterate over each algorithm
    for currentExperimentList in experimentListList:
        loadingBar = False

        print('Graph used in the experiment:')
        misc.graphInfo(G)

        coords = []

        # Iterate over each parameter set for an algorithm
        for experiments in currentExperimentList:

            iters = len(experiments)
            totRecolors = 0
            totColors = 0
            totTime = 0
            i = 0

            # Iterate over each identical iteration of the algorithm, to average results out
            for experiment in experiments:

                recolors = 0
                colors = 0

                if len(experiment) == 3:
                    random.seed(experiment[2])
                else:
                    random.seed()

                print("")
                print("------------------------------------------------------------------")
                print(str(experiment[1]) + ":")
                iterator = misc.UpdateIterator(experiment[0], updates)

                
                i += 1
                print("Iteration " + str(i))
                timer = 0
                verifytimer = 0
                colortimer = 0
                countertimer = 0
                totalUpdates = len(updates)
                done = 0
                barLength = 64

                if loadingBar:
                    #line to be overwritten
                    print("")

                while True:
                    
                    if loadingBar:
                        #overwrite previously printed line
                        percentageDone = int(done/totalUpdates*barLength)
                        b = ("[" + percentageDone * "+" + (barLength - percentageDone) * "-" + "]")
                        sys.stdout.write('\r'+b)
                        sys.stdout.flush()

                    timestamp = time.perf_counter()
                    # if not misc.verifyColoring(iterator.algo.G, iterator.algo.getColoring()):
                    #     raise ValueError('The coloring produced is incorrect')
                    verifytimer += time.perf_counter() - timestamp
                    timestamp = time.perf_counter()
                    c1 = iterator.algo.getColoring()
                    if iterator.runUpdate(1):
                        timer += time.perf_counter() - timestamp
                        timestamp = time.perf_counter()
                        c2 = iterator.algo.getColoring()
                        colortimer += time.perf_counter() - timestamp
                        timestamp = time.perf_counter()
                        recolors += misc.numberOfRecolors(c1, c2)
                        colors += misc.numberOfColors(c2)
                        countertimer += time.perf_counter() - timestamp
                        done += 1
                    else:
                        #print("all updates finished")
                        break

                #timer = time.perf_counter() - timer
                totTime += timer

                if loadingBar:
                    #overwrite previously printed line
                    b = ("")
                    sys.stdout.write('\r'+b)
                    sys.stdout.flush()

                avgRecolors = float(recolors)/len(updates)
                avgColors = float(colors)/len(updates)


                if (type == 'ratio'):
                    totRecolors += float(recolors)/totStaticRecolors
                    b = "Ratio of recolors: " + str(float(recolors)/totStaticRecolors)
                else:
                    totRecolors += avgRecolors
                    b = "Average number of recolors per update: " + str(avgRecolors)
                b = b + (barLength + 2 - len(b)) * " "
                print(b)

                if (type == 'end'):
                    totColors += misc.numberOfColors(iterator.algo.getColoring())
                    print("Total number of colors used at the end: " + str(misc.numberOfColors(iterator.algo.getColoring())))
                elif (type == 'average'):
                    totColors += avgColors
                    print("Average number of colors used: " + str(avgColors))
                elif (type == 'ratio'):
                    totColors += float(colors)/totStaticColors
                    print("Ratio of number of colors used: " + str(float(colors)/totStaticColors))


                print("Time taken on algorithm : " + str(timer) + " seconds")
                print("Time taken on verifying coloring: " + str(verifytimer) + " seconds")
                print("Time taken on counting colors and recolors: " + str(countertimer) + " seconds")
                print("Time taken on fetching the coloring: " + str(colortimer) + " seconds")
                #print("Average elementary operations : " + str(iterator.avgElemCounter))
                print("------------------------------------------------------------------")
            coords.append([totRecolors/iters, totColors/iters, totTime/iters])

        print("End of experiment list reached")
        coordinateLists.append(coords)
        # Save pickled coordinateLists
        with open('experiments/' + str(name)+' coordinateLists.pkl', 'wb') as f:
            pickle.dump(coordinateLists, f)


In [ ]:
runGraphExperiments(experimentListList, name, 'average')

In [ ]:
#FIX ALL DATA
names = ['SizeTestSmall']
for name in names:
    # Load pickled coordinateLists
    with open('experiments/' + str(name)+' coordinateLists.pkl', 'rb') as f:
        coordinateLists = pickle.load(f)

    for coordinateList in coordinateLists:
        for coordinate in coordinateList:
            coordinate[0] = coordinate[0]/4
            coordinate[1] = coordinate[1]/4

    # Save pickled coordinateLists
    with open('experiments/' +str(name)+' coordinateLists.pkl', 'wb') as f:
        pickle.dump(coordinateLists, f)
    

In [ ]:
name = 'SizeTestLarge'

In [ ]:
# Save pickled coordinateLists
with open('experiments/' +str(name)+' coordinateLists.pkl', 'wb') as f:
    pickle.dump(coordinateLists, f)

In [ ]:
# Load pickled coordinateLists
with open('experiments/' + str(name)+' coordinateLists.pkl', 'rb') as f:
    coordinateLists = pickle.load(f)

In [ ]:
coordinateLists

In [ ]:
plt.close()

algoNames = ['Random Warm Up', 'Small Bucket', 'Big Bucket', 'Static Dynamic', 'Static Simple', 'DC-Simple', 'DC-Orient']
colorDict = matplotlib.colors.CSS4_COLORS
colors = [colorDict.get('mediumpurple'), colorDict.get('palegreen'), colorDict.get('gold'), colorDict.get('cyan'), colorDict.get('peru'), colorDict.get('firebrick'), colorDict.get('navy')]
patches = []

# Quiver lists for arrows
qx = []
qy = []
qdx = []
qdy = []


j = 0
for list in coordinateLists:

    color = colors[j]
    patch = matplotlib.patches.Patch(color=color, label=algoNames[j])
    patches.append(patch)
    
    xcoords = [i[0] for i in list]
    ycoords = [i[1] for i in list]
    

    # Original line plot
    plt.plot(xcoords, ycoords, c=color, linewidth=2)

    # Points plot
    if (j != 0 and j != len(coordinateLists)-1):
        plt.scatter(xcoords, ycoords, c=color, s=5)
    else:
        plt.scatter(xcoords, ycoords, c=color, s=50)

    # Arrows
    if (len(list) > 1):
        # Pick arrow section
        x = xcoords[0:4]
        y = ycoords[0:4]

        # Calculate arrow position and direction
        u = x[-1]-x[0]
        v = y[-1]-y[0]
        pos_x = (x[0]*3+x[1])/4     #Weighted
        pos_y = (y[0]*3+y[1])/4     #Weighted
        norm = np.sqrt(u**2+v**2) 

        # Quiver info append
        qx.append(pos_x)
        qy.append(pos_y)
        qdx.append(u/norm)
        qdy.append(v/norm)

    j += 1

# Adjust labels depending on type of experiment
# Draw arrows and labels
plt.quiver(qx, qy, qdx, qdy, color=colors[1:-1], angles="xy", zorder=5, pivot="mid", headlength=4, headaxislength=4)
plt.xlabel("Average Recolors")
plt.ylabel("Average Colors")
plt.legend(handles=patches)
plt.show()

In [ ]:
plt.close()

algoNames = ['Random Warm Up', 'Small Bucket', 'Big Bucket', 'Static Dynamic', 'Static Simple', 'DC-Simple', 'DC-Orient']
colorDict = matplotlib.colors.CSS4_COLORS
colors = [colorDict.get('mediumpurple'), colorDict.get('palegreen'), colorDict.get('gold'), colorDict.get('cyan'), colorDict.get('peru'), colorDict.get('firebrick'), colorDict.get('navy')]
patches = []

# Quiver lists for arrows
qx = []
qy = []
qdx = []
qdy = []

j = 0
for list in coordinateLists:

    color = colors[j]
    patch = matplotlib.patches.Patch(color=color, label=algoNames[j])
    patches.append(patch)
    
    xcoords = [i[0] for i in list]
    ycoords = [i[2] for i in list]

    # Original line plot
    plt.plot(xcoords, ycoords, c=color, linewidth=2)

    # Points plot
    if (j != 0 and j != len(coordinateLists)-1):
        plt.scatter(xcoords, ycoords, c=color, s=5)
    else:
        plt.scatter(xcoords, ycoords, c=color, s=50)

    # Arrows
    if (j != 0 and j != len(coordinateLists)-1):
        # Pick arrow section
        x = xcoords[0:4]
        y = ycoords[0:4]

        # Calculate arrow position and direction
        u = x[-1]-x[0]
        v = y[-1]-y[0]
        pos_x = (x[0]*3+x[1])/4     #Weighted
        pos_y = (y[0]*3+y[1])/4     #Weighted
        norm = np.sqrt(u**2+v**2) 

        # Quiver info append
        qx.append(pos_x)
        qy.append(pos_y)
        qdx.append(u/norm)
        qdy.append(v/norm)

    j += 1

# Adjust labels depending on type of experiment
# Draw arrows and labels
plt.quiver(qx, qy, qdx, qdy, color=colors[1:-1], angles="xy", zorder=5, pivot="mid", headlength=4, headaxislength=4)
plt.xlabel("Average Recolors")
plt.ylabel("Total Time Taken (s)")
plt.legend(handles=patches)
plt.show()

# Full Plot Export

In [ ]:
names = [
 'SizeTestSmall', 'SizeTestMedium', 'SizeTestLarge',
 'DensityTestSmall', 'DensityTestMedium' ,'DensityTestLarge',
 'StreamRandom', 'StreamDecaying',
 'GraphDegreeFair', 'GraphDegreeSkewedAdditive', 'GraphDegreeSkewedSquared', 'GraphDegreeSingleHighDegree',
 'UpdateSequenceRandom', 'UpdateSequenceExpanding', 'UpdateSequencePrioritized'
 ]
 
for name in names:

    # Load pickled coordinateLists
    with open('experiments/' + str(name)+' coordinateLists.pkl', 'rb') as f:
        coordinateLists = pickle.load(f)

    #FIRST PLOT
    for collapse in range(1):
        plt.close()

        algoNames = ['Random Warm Up', 'Small Bucket', 'Big Bucket', 'Static Dynamic', 'Static Simple', 'DC-Simple', 'DC-Orient']
        colorDict = matplotlib.colors.CSS4_COLORS
        colors = [colorDict.get('mediumpurple'), colorDict.get('palegreen'), colorDict.get('gold'), colorDict.get('cyan'), colorDict.get('peru'), colorDict.get('firebrick'), colorDict.get('navy')]
        patches = []

        # Quiver lists for arrows
        qx = []
        qy = []
        qdx = []
        qdy = []


        j = 0
        for list in coordinateLists:

            color = colors[j]
            patch = matplotlib.patches.Patch(color=color, label=algoNames[j])
            patches.append(patch)
            
            xcoords = [i[0] for i in list]
            ycoords = [i[1] for i in list]
            

            # Original line plot
            plt.plot(xcoords, ycoords, c=color, linewidth=2)

            # Points plot
            if (j != 0 and j != len(coordinateLists)-1):
                plt.scatter(xcoords, ycoords, c=color, s=5)
            else:
                plt.scatter(xcoords, ycoords, c=color, s=50)

            # Arrows
            if (j != 0 and j != len(coordinateLists)-1):
                # Pick arrow section
                x = xcoords[0:4]
                y = ycoords[0:4]

                # Calculate arrow position and direction
                u = x[-1]-x[0]
                v = y[-1]-y[0]
                pos_x = (x[0]*3+x[1])/4     #Weighted
                pos_y = (y[0]*3+y[1])/4     #Weighted
                norm = np.sqrt(u**2+v**2) 

                # Quiver info append
                qx.append(pos_x)
                qy.append(pos_y)
                qdx.append(u/norm)
                qdy.append(v/norm)

            j += 1

        # Adjust labels depending on type of experiment
        # Draw arrows and labels
        plt.quiver(qx, qy, qdx, qdy, color=colors[1:-1], angles="xy", zorder=5, pivot="mid", headlength=4, headaxislength=4)
        plt.xlabel("Average Recolors")
        plt.ylabel("Average Colors")
        plt.legend(handles=patches)
        plt.savefig('experiments/images/'+name+".png")

    #SECOND PLOT
    for collapse in range(1):
        plt.close()
        algoNames = ['Random Warm Up', 'Small Bucket', 'Big Bucket', 'Static Dynamic', 'Static Simple', 'DC-Simple', 'DC-Orient']
        colorDict = matplotlib.colors.CSS4_COLORS
        colors = [colorDict.get('mediumpurple'), colorDict.get('palegreen'), colorDict.get('gold'), colorDict.get('cyan'), colorDict.get('peru'), colorDict.get('firebrick'), colorDict.get('navy')]
        patches = []

        # Quiver lists for arrows
        qx = []
        qy = []
        qdx = []
        qdy = []

        j = 0
        for list in coordinateLists:

            color = colors[j]
            patch = matplotlib.patches.Patch(color=color, label=algoNames[j])
            patches.append(patch)
            
            xcoords = [i[0] for i in list]
            ycoords = [i[2] for i in list]

            # Original line plot
            plt.plot(xcoords, ycoords, c=color, linewidth=2)

            # Points plot
            if (j != 0 and j != len(coordinateLists)-1):
                plt.scatter(xcoords, ycoords, c=color, s=5)
            else:
                plt.scatter(xcoords, ycoords, c=color, s=50)

            # Arrows
            if (j != 0 and j != len(coordinateLists)-1):
                # Pick arrow section
                x = xcoords[0:4]
                y = ycoords[0:4]

                # Calculate arrow position and direction
                u = x[-1]-x[0]
                v = y[-1]-y[0]
                pos_x = (x[0]*3+x[1])/4     #Weighted
                pos_y = (y[0]*3+y[1])/4     #Weighted
                norm = np.sqrt(u**2+v**2) 

                # Quiver info append
                qx.append(pos_x)
                qy.append(pos_y)
                qdx.append(u/norm)
                qdy.append(v/norm)

            j += 1

        # Adjust labels depending on type of experiment
        # Draw arrows and labels
        plt.quiver(qx, qy, qdx, qdy, color=colors[1:-1], angles="xy", zorder=5, pivot="mid", headlength=4, headaxislength=4)
        plt.xlabel("Average Recolors")
        plt.ylabel("Total Time Taken (s)")
        plt.legend(handles=patches)
        plt.savefig('experiments/images/'+name+'Time.png')

# Full Graph Experiment

In [ ]:
names = ['SizeTestSmall']
for name in names:
    #Imports
    %matplotlib widget
    import pandas as pd
    import numpy as np
    import csv
    import matplotlib
    import matplotlib.pyplot as plt
    import scipy
    import datetime
    import networkx as nx
    import misc
    import math
    import sys
    import random
    import SmallBucket
    import BigBucket
    import RandomWarmUp1
    import xyz_RandomWarmUp2 as RandomWarmUp2
    import StaticDynamic
    import DcOrient
    import time
    import xyz_DcOrientRandomSimple as DcOrientRandomSimple
    import DcSimple
    import StaticSimple
    import xyz_BigBucketSimple as BigBucketSimple
    import xyz_SmallBucketSimple as SmallBucketSimple
    import xyz_StaticRandom as StaticRandom
    import pickle

    # Load graph data
    gd = misc.loadGraphData(name)

    Ge = gd.G.copy()

    updates = gd.updates
    staticColors = gd.staticColors
    staticRecolors = gd.staticRecolors
    totStaticColors = gd.colorSum
    totStaticRecolors = gd.recolorSum

    # Get final state of graph in local memory as well
    G = gd.G.copy()
    for update in updates:
        if update[1]:
            G.add_edge(update[0][0], update[0][1])
        else:
            G.remove_edge(update[0][0], update[0][1])

    e = [['SmallBucket', 1, 30],
    ['BigBucket', 1, 30],
    ['StaticDynamic', 1, 200],
    ['StaticSimple', 1, 200],
    ['DcSimple', 0.4, 1]]

    iterations = 3

    coordinateLists = []
    experimentListList = []

    # Add a single iteration for the random warm up
    experimentList = []
    tempList = []
    for j in range(0, iterations):
        tempList.append([RandomWarmUp1.WarmUp1Algo(Ge, maxDegreeBound=None), 'Random Warm-Up'])
    experimentList.append(tempList)
    experimentListList.append(experimentList)


    for exp in e:
        # Add different algorithms one by one by changing parameters
        experimentList = []
        algo = exp[0]
        min = exp[1]
        max = exp[2]
        r = 100            # Number of experiments per algorithm
        current = min
        lastCurrent = -100
        if algo == 'DcSimple':  # TEMP
             r = 10
        for i in range(0, r):
            if algo == 'SmallBucket':
                d = int(current)
                if d != int(lastCurrent):
                    experimentList.append([[SmallBucket.SmallBucketAlgo(d, Ge), 'Small-Bucket algorithm (d = '+str(d)+')', 10]])
            elif algo == 'BigBucket':
                d = int(current)
                if d != int(lastCurrent):
                    experimentList.append([[BigBucket.BigBucketAlgo(d, Ge), 'Big-Bucket algorithm (d = '+str(d)+')', 10]])
            elif algo == 'BigBucketSimple':
                d = exp[3] #temporary
                p = current
                experimentList.append([[BigBucketSimple.BigBucketSimpleAlgo(d, p, Ge), 'Big-Bucket Simple algorithm (p = '+str(p)+')']])
            elif algo == 'SmallBucketSimple':
                d = 10
                p = current
                experimentList.append([[SmallBucketSimple.SmallBucketSimpleAlgo(d, p, Ge), 'Small-Bucket Simple algorithm (p = '+str(p)+')']])
            elif algo == 'StaticDynamic':
                l = int(current)
                if l != int(lastCurrent):
                    tempList = []
                    for j in range(0, iterations):
                        tempList.append([StaticDynamic.StaticDynamicAlgo(Ge, l, dynamicReset=False), 'Static-Dynamic algorithm (l = '+str(l)+')'])
                    experimentList.append(tempList)
            elif algo == 'StaticSimple':
                l = int(current)
                if l != int(lastCurrent):
                    tempList = []
                    for j in range(0, iterations):
                        tempList.append([StaticSimple.StaticSimpleAlgo(Ge, l=l), 'Static-Simple (l = '+str(l)+')'])
                    experimentList.append(tempList)
            elif algo == 'DcSimple':
                p = current
                tempList = []
                for j in range(0, iterations):
                    tempList.append([DcSimple.DcSimpleAlgo(Ge, p=p), 'DC-Simple (p = '+str(p)+')'])
                experimentList.append(tempList)
            elif algo == 'StaticRandom':
                p = current
                tempList = []
                for j in range(0, iterations):
                    tempList.append([StaticRandom.StaticRandomAlgo(Ge, p=p), 'Static Random (p = '+str(p)+')'])
                experimentList.append(tempList)

            lastCurrent = current
            
            if (algo == 'DcSimple'):
                current = current + ((max-min)/10)  #TEMP
            else:
                current = current + ((max-min)/100)

        experimentListList.append(experimentList)

    # Append a single iteration of DC-Orient
    experimentList = []
    experimentList.append([[DcOrient.DcOrientAlgo(Ge), 'DC-Orient', 10]])
    experimentListList.append(experimentList)

    runGraphExperiments(experimentListList, name, 'average')

    # Save pickled coordinateLists
    with open('experiments/' + str(name)+' coordinateLists.pkl', 'wb') as f:
        pickle.dump(coordinateLists, f)

    print(str(name) + ' experiment done!')

# Results

In [ ]:
runExperiments(experiments)